In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import subprocess

def frequent_itemsets(filename, min_support, type='M'):
    def split_itemset(str):
        itemsets = str.split()
        if (len(itemsets) < 2):
            return None
        
        freq = itemsets[-1]
        freq = freq[1:-1]
    
        return int(freq), list(map(int, itemsets[0:-1]))
    
    p1 = subprocess.Popen(["./lcm", '{}f'.format(type), filename, str(min_support), "-"], stdout=subprocess.PIPE)
    
    output = p1.communicate(str.encode("utf-8"))[0]
    output = output.decode('utf-8')
    itemsets = output.split('\n')
    
    itemsets = list(map(split_itemset, itemsets))
    itemsets = list(filter(None.__ne__, itemsets))
    return itemsets

In [ ]:
binary_filepath = "mnist-binary.txt"

data = np.loadtxt(binary_filepath)
X = data[:, :-1]
y = data[:, -1]

In [ ]:
digit_0 = X[y == 0]
# digit_0 = digit_0[:100]
temp_filepath = "temp.txt"
os.remove(temp_filepath)
with open(temp_filepath, "a") as file:
    
    for i in range(digit_0.shape[0]):
        trans = digit_0[i]
        item_list = np.nonzero(trans)[0].tolist()
        l = " ".join(str(item) for item in item_list)
        file.write(l)
        file.write("\n")

res = frequent_itemsets(temp_filepath, 5000)
print("Maximal itemset count : %s" % len(res))

In [ ]:
# Calculating disjoint maximal frequent item set
disjoint_set_list = []
for frequency, s in res:
    s_set = set(s)
    to_add = True
    for _, a_set in disjoint_set_list:
        if len(a_set & s_set) > 0:
            to_add = False
            break
    if to_add:
        disjoint_set_list.append((frequency, s_set))
        
print(len(disjoint_set_list))

In [ ]:
X_b = X.astype(bool)

def calculate_confidence_for_single_frequent_set(index):

    sample = list(disjoint_set_list[index][1])

    XY_mask = np.zeros(784).astype(bool)
    XY_mask[sample] = True
    # plt.imshow(XY_mask.reshape(28, 28), cmap='gray')
    # plt.show()

    X_support = res[0][0]
    C = np.zeros(784) # Confidence matrix

    for i in range(784):

        X_set = sample[:]
        if i not in X_set:
            X_set.append(i)

        XY_support = np.sum( np.all(X_b[:, X_set], axis = 1))
        C[i] = XY_support / X_support

    # plt.imshow(C.reshape(28, 28), cmap='gray')
    # plt.show()
    
    return C

no_frequent_disjoint_set = len(disjoint_set_list)
C = np.empty((no_frequent_disjoint_set, 784))

print( no_frequent_disjoint_set)
for i in range(no_frequent_disjoint_set):
    C[i] = calculate_confidence_for_single_frequent_set(i);
    print(i)

In [ ]:
from sklearn.preprocessing import StandardScaler

# Now I have confidence matrix, compute missing values
sample = X_b[16].copy()
plt.imshow(sample.reshape(28, 28), cmap='gray')
plt.show()

sample[560 - 28:] = False
plt.imshow(sample.reshape(28, 28), cmap='gray')
plt.show()

sample_r = np.zeros(784)
for i in range(no_frequent_disjoint_set):
    frequent_set = list(disjoint_set_list[i][1])
    if np.any(sample[frequent_set]):
        sample_r += C[i]

sample_r = sample_r.reshape(-1, 1)
plt.imshow(sample_r.reshape(28, 28), cmap='gray')
plt.show()

#print(sample_r)
scaler = StandardScaler()
sample_r = scaler.fit_transform(sample_r)

#print(sample_r)
sample_r = np.array(sample_r > 1.2, dtype=int)
plt.imshow(sample_r.reshape(28, 28), cmap='gray')
plt.show()

sample_r = sample_r.ravel()
sample_r[:560 - 28] = sample[:560 - 28]
plt.imshow(sample_r.reshape(28, 28), cmap='gray')
plt.show()